# Before and After 2006 Fire Management Analysis

This notebook reproduces and extends analysis from **Podur (2009)** examining changes in forest fire patterns in Ontario before and after policy changes implemented around 2005-2006. The analysis investigates how fire management zone designations and seasonal patterns influence fire size distributions and area burned.

## Background

Podur (2009) analyzed changes in Ontario's forest fire regime, particularly examining the impact of different fire management strategies across zones. This notebook builds on that work by:

1. **Spatial Analysis**: Joining fire occurrence data with historical fire management zones to understand regional differences
2. **Temporal Analysis**: Comparing fire patterns before and after 2005 to identify potential policy impacts
3. **Statistical Modeling**: Fitting Weibull distributions to fire size data and testing against exponential distributions
4. **Seasonal Patterns**: Examining how fire characteristics vary across seasons (Spring, Summer, Fall, Winter)

## Key Components

- **Fire Management Zones**: Analysis distinguishes between "Intensive Measured" zones (Great Lakes/St. Lawrence, Boreal, Northern Boreal) and "Extensive" zones (Hudson Bay)
- **Large Fire Focus**: Primary analysis on fires >100 hectares, which represent the most significant fire events
- **Distribution Analysis**: Fitting statistical distributions to understand fire size patterns and their changes over time
- **Moment Analysis**: Calculating first, second, and third moments of fire size distributions to characterize their shape and variability

## Data Sources

- National Fire Database (NFDB) point data for fire occurrences
- Historical Fire Management Zones shapefile for spatial classification
- Analysis period: 1976-present, with comparison point at 2005

## Imports and data cleaning

In [ ]:
import geopandas as gpd
import numpy as np
from scipy.stats import weibull_min, chi2

# Read the fire data
gdf = gpd.read_file("nfdb/NFDB_point_20250519.shp")

# read in the historical fire management zones
zones = gpd.read_file("historical_fire_zones\Historical_Fire_Management_Zones.shp")

# Ensure both GeoDataFrames use the same CRS
gdf = gdf.to_crs(zones.crs)

# join the two GeoDataFrames
data = gpd.sjoin(gdf, zones, how="inner", predicate="within")

# drop the data in FMZ_DESIGN == 'Parks Zone'
data = data[data.FMZ_DESIGN != "Parks Zone"]

data.NFDBFIREID.duplicated().sum()

# show me the rows where NFDBFIREID is duplicated
duplicates = data[data.NFDBFIREID.duplicated(keep=False)]

# remove the duplicates
data = data[~data.NFDBFIREID.duplicated(keep="first")]

# select only rows with CAUSE = N or U

data = data[(data.CAUSE == "N") | (data.CAUSE == "U")]

# drop rows where the REP_DATE is null
joined = data[data.REP_DATE.notnull()]

# make a new column for whether the SIZE_HA is greater than 100\
joined["SIZE_HA_100"] = joined["SIZE_HA"].apply(lambda x: True if x > 100 else False)

# remove the rows with YEAR before 1975
joined = joined[joined.YEAR >= 1976]

# make dictionary to change values of FMZ_DESIGN
fmz_dict = {
    "Hudson Bay Zone": "Extensive",
    "Great Lakes/St. Lawrence Zone": "Intensive Measured",
    "Boreal Zone": "Intensive Measured",
    "Northern Boreal Zone": "Intensive Measured"
}

# make a new column in the joined GeoDataFrame
joined["FMZ_ZONE"] = joined["FMZ_DESIGN"].map(fmz_dict)

season_dict = {1: "Winter",
               2: "Winter",
               3: "Spring",
                4: "Spring",
                5: "Spring",
                6: "Summer",
                7: "Summer",
                8: "Summer",
                9: "Fall",
                10: "Fall",
                11: "Fall",
                12: "Winter"}

# make a new column for the season based on the column called "MONTH"
joined["SEASON"] = joined["MONTH"].map(season_dict)


# make a new column based on the column year which is whether it is after 2006 or not
def after_2005(year):
    if year > 2005:
        return "After 2005"
    else:
        return "Before 2005"

joined["AFTER_2005"] = joined["YEAR"].apply(after_2005)

# select the columns that we want to keep

joined = joined[['YEAR', 'REP_DATE', 'SIZE_HA', 'FMZ_ZONE', 'SEASON', 'AFTER_2005', 'SIZE_HA_100']]

In [ ]:
# make a table for the total area burned by year with another column as the number of fires and the number of fires greater than 100 ha
joined_grouped = joined.groupby(['YEAR']).agg(
    {'SIZE_HA': 'sum'}).reset_index()

# now make the same columns by for the fires in the Intensive Measured zone
joined_grouped_intensive = joined[joined.FMZ_ZONE == "Intensive Measured"].groupby(['YEAR']).agg(
    {'REP_DATE':'count', 'SIZE_HA_100': 'sum'}).reset_index()

# rename the columns
joined_grouped_intensive = joined_grouped_intensive.rename(
    columns={'REP_DATE': 'Number of Fires in the IM Zones', 'SIZE_HA_100': 'Number of Large Fires in the IM Zones'})

# do the same for the Extensive zone
joined_grouped_extensive = joined[joined.FMZ_ZONE == "Extensive"].groupby(['YEAR']).agg(
    {'REP_DATE':'count', 'SIZE_HA_100': 'sum'}).reset_index()

# rename the columns
joined_grouped_extensive = joined_grouped_extensive.rename(
    columns={'REP_DATE': 'Number of Fires in the Ext Zones', 'SIZE_HA_100': 'Number of Large Fires in the Ext Zones'})

# now merge the two dataframes
joined_grouped = pd.merge(joined_grouped, joined_grouped_intensive, on='YEAR', how='left')

# now merge the two dataframes

joined_grouped = pd.merge(joined_grouped, joined_grouped_extensive, on='YEAR', how='left')

joined_grouped

,YEAR,SIZE_HA,Number of Fires in the IM Zones,Number of Large Fires in the IM Zones,Number of Fires in the Ext Zones,Number of Large Fires in the Ext Zones
0,1976,457468.201016,1849,54,106,52
1,1977,387769.998153,474,12,58,32
2,1978,3655.700003,161,2,5,2
3,1979,59955.099868,627,16,29,10
4,1980,262433.598454,725,40,42,13
5,1981,137685.701306,412,10,64,38
6,1982,1683.499995,236,2,7,1
7,1983,426824.698229,946,29,113,50
8,1984,107222.703304,266,2,26,13
9,1985,142.400001,161,0,4,0


In [ ]:
# I want to fit a weibull distribution to the data by the before or after 2005, and also the season

# make a functoin that fit the weibull distribution to the data

def fit_weibull_exp(data, col):
    # fit the weibull distribution to the data
    params = weibull_min.fit(data[col], floc=0)
    # get the shape, scale and location parameters
    shape, loc, scale = params
    # get the exponential mean too 
    mean = data[col].mean()
    # Log-likelihood for Weibull
    ll_weibull = np.sum(weibull_min.logpdf(data[col], shape, loc=loc, scale=scale))
    # Fit Exponential (Weibull with shape=1)
    exp_scale = data[col].mean()
    ll_exp = np.sum(weibull_min.logpdf(data[col], 1, loc=0, scale=exp_scale))
    # Likelihood Ratio statistic
    LR = 2 * (ll_weibull - ll_exp)
    p_value = 1 - chi2.cdf(LR, df=1)


    return shape, scale, mean, p_value
# get only the data with HA_100 == True

joined_100 = joined[joined.SIZE_HA_100 == True]

# now group the data by the BEFORE_2005 and SEASON columns
joined_grouped_weibull = joined_100.groupby(['AFTER_2005', 'FMZ_ZONE','SEASON'])

# apply the fit_weibull function to each group
weibull_params = joined_grouped_weibull.apply(lambda x: fit_weibull_exp(x, 'SIZE_HA')).reset_index()

# turn the column called 0 onto two columns called shape and scale
weibull_params[['shape', 'scale', 'mean', 'p_value']] = pd.DataFrame(weibull_params[0].tolist(), index=weibull_params.index)

# drop the zero column
weibull_params = weibull_params.drop(columns=[0])

# sort by the AFter_2005 and then the FMZ_ZONE
weibull_params = weibull_params.sort_values(by=['AFTER_2005', 'FMZ_ZONE'])

weibull_params

C:\Users\LiW\AppData\Local\Temp\ipykernel_25856\4070019939.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weibull_params = joined_grouped_weibull.apply(lambda x: fit_weibull_exp(x, 'SIZE_HA')).reset_index()


,AFTER_2005,FMZ_ZONE,SEASON,shape,scale,mean,p_value
0,After 2005,Extensive,Fall,1.080143,850.619284,823.724999,7.781803e-01
1,After 2005,Extensive,Spring,0.454365,3684.471332,10334.739991,3.754156e-03
2,After 2005,Extensive,Summer,0.596554,1877.513563,3395.532135,0.000000e+00
3,After 2005,Intensive Measured,Fall,0.747415,995.233438,1252.267281,7.783966e-04
4,After 2005,Intensive Measured,Spring,0.657240,1435.478552,2110.528574,2.188149e-03
5,After 2005,Intensive Measured,Summer,0.525183,2486.149237,5846.153486,0.000000e+00
6,Before 2005,Extensive,Fall,0.668010,1299.831815,1786.674955,2.398048e-01
7,Before 2005,Extensive,Spring,0.605247,5731.670769,8650.091411,1.026768e-09
8,Before 2005,Extensive,Summer,0.622350,2128.234047,3424.596434,0.000000e+00
9,Before 2005,Intensive Measured,Fall,0.542146,2757.900114,5241.139920,2.444071e-03


In [36]:
# group by the AFTER_2005, FMZ_ZONE and SEASON columns
joined_grouped_moments = joined_100.groupby(['AFTER_2005', 'FMZ_ZONE','SEASON'])

# get the mean and second and third moments of SIZE_HA
def get_moments(data, col):
    mean = data[col].mean()
    second_moment = np.mean((data[col] - mean) ** 2)
    third_moment = np.mean((data[col] - mean) ** 3)
    return mean, second_moment, third_moment

# apply the get_moments function to each group
moments = joined_grouped_moments.apply(lambda x: get_moments(x, 'SIZE_HA')).reset_index()

# turn the column called 0 onto three columns called mean, second_moment and third_moment

moments[['Average Fire Size', 'Second Moment (Fire Size)', 'Third Moment (Fire Size)']] = pd.DataFrame(moments[0].tolist(), index=moments.index)

# drop the zero column
moments = moments.drop(columns=[0])

moments

C:\Users\LiW\AppData\Local\Temp\ipykernel_25856\107362128.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  moments = joined_grouped_moments.apply(lambda x: get_moments(x, 'SIZE_HA')).reset_index()


,AFTER_2005,FMZ_ZONE,SEASON,Average Fire Size,Second Moment (Fire Size),Third Moment (Fire Size)
0,After 2005,Extensive,Fall,823.724999,6.454888e+05,7.349141e+08
1,After 2005,Extensive,Spring,10334.739991,3.655922e+08,1.045734e+13
2,After 2005,Extensive,Summer,3395.532135,1.138104e+08,1.083856e+13
3,After 2005,Intensive Measured,Fall,1252.267281,5.551733e+06,5.564918e+10
4,After 2005,Intensive Measured,Spring,2110.528574,1.868859e+07,2.902236e+11
5,After 2005,Intensive Measured,Summer,5846.153486,3.363923e+08,4.319046e+13
6,Before 2005,Extensive,Fall,1786.674955,6.165157e+06,1.726593e+10
7,Before 2005,Extensive,Spring,8650.091411,1.625309e+08,3.741265e+12
8,Before 2005,Extensive,Summer,3424.596434,6.164644e+07,2.422673e+12
9,Before 2005,Intensive Measured,Fall,5241.139920,7.915375e+07,1.052403e+12


In [38]:
# sum up the area burned for each year and the number of fires and include the AFTER_2005 and FMZ_ZONE and season
joined_grouped_moments = joined_100.groupby(['YEAR', 'AFTER_2005', 'FMZ_ZONE','SEASON']).agg(
    {'SIZE_HA': 'sum', 'REP_DATE':'count'}).reset_index()

# make a column for the second moment
joined_grouped_moments['second_moment'] = joined_grouped_moments['SIZE_HA'] ** 2

# make a column for the third moment
joined_grouped_moments['third_moment'] = joined_grouped_moments['SIZE_HA'] ** 3


joined_grouped_moments = joined_grouped_moments.groupby(['AFTER_2005', 'FMZ_ZONE','SEASON']).agg(
    {'SIZE_HA': 'mean', 'REP_DATE':'mean', 'second_moment':'mean', 'third_moment':'mean'}).reset_index()

# rename the SIZE_HA to the mean area burned and the REP_DATE to the mean number of fires

joined_grouped_moments = joined_grouped_moments.rename(
    columns={'SIZE_HA': 'Mean Area Burned', 'REP_DATE': 'Mean Number of Fires', 'second_moment': 'Second Moment (Area Burned)', 'third_moment': 'Third Moment (Area Burned)'})

# now merge the two dataframes
joined_grouped_moments_merge = pd.merge(moments, joined_grouped_moments, on=['AFTER_2005', 'FMZ_ZONE','SEASON'], how='left')

joined_grouped_moments_merge

,AFTER_2005,FMZ_ZONE,SEASON,Average Fire Size,Second Moment (Fire Size),Third Moment (Fire Size),Mean Area Burned,Mean Number of Fires,Second Moment (Area Burned),Third Moment (Area Burned)
0,After 2005,Extensive,Fall,823.724999,6.454888e+05,7.349141e+08,1647.449998,2.000000,4.545772e+06,1.527435e+10
1,After 2005,Extensive,Spring,10334.739991,3.655922e+08,1.045734e+13,17224.566652,1.666667,8.129670e+08,4.003309e+13
2,After 2005,Extensive,Summer,3395.532135,1.138104e+08,1.083856e+13,63383.266514,18.666667,9.589971e+09,2.089060e+15
3,After 2005,Intensive Measured,Fall,1252.267281,5.551733e+06,5.564918e+10,8609.337558,6.875000,4.179293e+08,2.380086e+13
4,After 2005,Intensive Measured,Spring,2110.528574,1.868859e+07,2.902236e+11,5540.137506,2.625000,7.223855e+07,1.219425e+12
5,After 2005,Intensive Measured,Summer,5846.153486,3.363923e+08,4.319046e+13,68326.918873,11.687500,1.648845e+10,4.945906e+15
6,Before 2005,Extensive,Fall,1786.674955,6.165157e+06,1.726593e+10,7146.699821,4.000000,5.107532e+07,3.650200e+11
7,Before 2005,Extensive,Spring,8650.091411,1.625309e+08,3.741265e+12,40367.093250,4.666667,4.563933e+09,7.184317e+14
8,Before 2005,Extensive,Summer,3424.596434,6.164644e+07,2.422673e+12,85492.603827,24.964286,1.280480e+10,2.305229e+15
9,Before 2005,Intensive Measured,Fall,5241.139920,7.915375e+07,1.052403e+12,52411.399200,10.000000,2.746955e+09,1.439717e+14


In [33]:
joined_grouped_moments

,AFTER_2005,FMZ_ZONE,SEASON,Mean Area Burned,Mean Number of Fires,Mean Second Moment,Mean Third Moment
0,After 2005,Extensive,Fall,1647.449998,2.000000,4.545772e+06,1.527435e+10
1,After 2005,Extensive,Spring,17224.566652,1.666667,8.129670e+08,4.003309e+13
2,After 2005,Extensive,Summer,63383.266514,18.666667,9.589971e+09,2.089060e+15
3,After 2005,Intensive Measured,Fall,8609.337558,6.875000,4.179293e+08,2.380086e+13
4,After 2005,Intensive Measured,Spring,5540.137506,2.625000,7.223855e+07,1.219425e+12
5,After 2005,Intensive Measured,Summer,68326.918873,11.687500,1.648845e+10,4.945906e+15
6,Before 2005,Extensive,Fall,7146.699821,4.000000,5.107532e+07,3.650200e+11
7,Before 2005,Extensive,Spring,40367.093250,4.666667,4.563933e+09,7.184317e+14
8,Before 2005,Extensive,Summer,85492.603827,24.964286,1.280480e+10,2.305229e+15
9,Before 2005,Intensive Measured,Fall,52411.399200,10.000000,2.746955e+09,1.439717e+14
